# Introduction

This program draws a "heat map" of COVID cases based on the Covid-19 dataset. The dataset includes information about
case counts over time. <br>
It relies on a pre-made country map JSON to make the Folium overlay, and a "better_names" csv which I wrote so that the
code wouldn't get too cluttered.

# Main Program

## Init

Imports standard libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import folium as fol
import pycountry as pc
from urllib.request import urlopen
from json import load

Loads data files on GitHub (for Colab)

In [ ]:
COVID_DATA_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/covid_19_data.csv"
BETTER_NAMES_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/better_names.csv"
MERGED_COUNTRIES_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/merged_countries.json"

# df = pd.read_csv(COVID_DATA_URL)
# bn = pd.read_csv(BETTER_NAMES_URL)
# ct = load(urlopen(MERGED_COUNTRIES_URL))

Loads Data Files (for execution locally/ GitHub).

In [ ]:
df = pd.read_csv("../Dat/covid_19_data.csv")
bn = pd.read_csv("../Dat/better_names.csv")
ct = load(open("../Dat/merged_countries.json"))


## -- Run --

Function which changes the poorly-named-countries into ones that PyCountry can recognize.

In [ ]:
def sanitize_csv(original, new):
    if new == "nil" or new is None:
        df["Country"] = df["Country"].str.replace(original, "", regex=True)
    else:
        df["Country"] = df["Country"].str.replace(original, new, regex=True)

Prepares the table for use by dropping unneeded columns and renaming an annoying one.


In [ ]:
df.drop(["SNo", "ObservationDate", "Recovered", "Last Update", "Deaths"], axis=1, inplace=True)
df.rename(columns = {"Country/Region": "Country"}, inplace=True)

"Sanitizes" the country names so that PyCountry will recognize them, then collapses them all together.

In [ ]:
for row in bn.itertuples(): sanitize_csv(row[1], row[2])

Groups by and finds the maximum value for each state (the entries are cumulative, so a single group would result in an
absurd amount of cases.)

In [ ]:
df = df.groupby(["Country", "Province/State"], as_index=False, dropna=False).aggregate({"Confirmed":"last"})

Groups the table again by country, finding the sum of all of the states.

In [ ]:
df = df.groupby(["Country"], as_index=False, dropna=False).aggregate({"Confirmed":"sum"})

Uses PyCountry to find the three-letter acronym for each country for use with Folium.

In [ ]:
for row in df["Country"]: df["Country"] = df["Country"].replace(row, pc.countries.search_fuzzy(row)[0].alpha_3)

Takes the log of all values in order to make a more meaningful map. <br>
Without this step, only three countries would be colored anything other than yellow.

In [ ]:
for row in df["Confirmed"]: df["Confirmed"] = df["Confirmed"].replace(row, np.log(row))
df.dropna(inplace=True) # Necessary to remove broken entries created by the above process.

Creates the Folium map.

In [ ]:
outbreak_map = fol.Map(location=[0, 0], zoom_start=0)

fol.Choropleth(
    name = "COVID Cases",
    geo_data = ct, # Polygonal data to draw the country map.
    data = df, # COVID case data
    columns = ["Country", "Confirmed"], # Column to match with the key, count-based column.
    key_on = "feature.id", # Establishes the key of the country JSON.
    fill_color = "YlOrRd", # Color scheme
    fill_opacity = 0.75,
    line_opacity = 0.25,
    nan_fill_opacity = 0,
    legend_name = "Confirmed Cases",
    highlight = True,
).add_to(outbreak_map)

Creates an overlay for the above folium map which displays the confirmed count.

In [ ]:
OverlayData = fol.features.GeoJson(
    ct,
    style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.2, 'weight': 0.2},
    highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1},
    control = False,
    tooltip = fol.features.GeoJsonTooltip(
        fields = ["name", "Confirmed"],
        aliases = ["Country: ", "Confirmed Cases: "],
        style = "background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
    )
)

outbreak_map.add_child(OverlayData)
outbreak_map.keep_in_front(OverlayData)
fol.LayerControl().add_to(outbreak_map)

## Map of the COVID Outbreak

In [ ]:
outbreak_map

# Project Tasks

## Stage 1 - Using Pandas

### 1.3 - Loading a Dataframe

Loads the dataframe again; the import call is commented out, as it was done earlier.

In [ ]:
# import pandas as pd
CSV_DATA = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/covid_19_data.csv"

# df = pd.read_csv(CSV_DATA)                # Loads data from GitHub (CoLab)
df = pd.read_csv("../Dat/covid_19_data.csv")# Loads local data (Local/Git)

Sanitizes the master dataframe (again) to clean up country names.

In [ ]:
df.rename(columns = {"Country/Region": "Country"}, inplace=True)
for row in bn.itertuples(): sanitize_csv(row[1], row[2])
df.rename(columns = {"Country": "Country/Region"}, inplace=True) # Resets title scheme back to original.

### 1.4 - Manipulation with Workflows

#### 1.4a - Datatypes

Displays the datatypes in the dataset.

In [ ]:
print("Data Types:\n", df.dtypes)

#### 1.4b - Top

Displays the top of the dataframe with .head()

In [ ]:
print("Top of Set:\n", df.head())

#### 1.4c - Summary

Creates an automatically-generated statistical summary of the dataframe.

In [ ]:
print("Summary:\n", df.describe())

## Stage 2 - Manipulating and Writing Data

### 2.4 - Adding/Editing Columns

Creates a new dataframe for this section.

In [ ]:
p2 = df.copy()

Combines "Province/State" and "Country/Region" into a new column ("Location") in the form "Province, Country".

In [ ]:
p2["Location"] = p2["Province/State"] + ", " + p2["Country/Region"]
p2

### 2.7a - Creating a Subset

Uses .groupby().agg() to narrow down the table into a more useful form. <br>
Dataset is filtered before other steps to speed up the process.

In [ ]:
# as_index = False so that the next stage is possible.
p2 = p2.groupby(["SNo", "ObservationDate", "Location", "Confirmed", "Recovered", "Deaths"],
                as_index=False).agg({"Recovered":"sum"})
p2

### 2.6 - set_index()

Sets the index to "SNo", which is the original index number for the dataset.

In [ ]:
p2.set_index(["SNo"], inplace=True)
p2

### 2.5 - A Filtering Operation

Filters the dataset so that only entries with a "Recovered" value greater than the mean will be included. <br>
This reduces the dataset by a factor of ten.

In [ ]:
p2 = p2[p2.Recovered > p2.Recovered.mean()]
p2

### 2.7b - to_csv()

Saves the filtered dataframe to "test_output.csv" in the working folder.

In [ ]:
p2.to_csv("test_output.csv")


## Stage 3 - Data Cleansing and Plotting

### Part 3.1 - NaN

#### 3.1.1 - isna()

Finds the number of missing values in the table by column.

In [ ]:
p3 = df.copy()
p3.isna().sum()

There are 62,045 missing values in the table; all located under the Province/State column.

#### 3.1.3 - dropna()

Drops all of the rows with null values. <br>
Displays a count of all remaining NaN values to show

In [ ]:
trunc = p3.dropna()
trunc.isna().sum().sum() # Total count of NaN values

#### 3.1.2 - fillna()

Instead of dropping the null values, this replaces them with "Undefined". <br>
Prints a row with a null value before/after the modification to show the change.

In [ ]:
print(p3["Province/State"][35])
p3.fillna("Undefined", inplace=True)
print(p3["Province/State"][35])
p3.isna().sum().sum() # Total count of NaN values.

### Part 3.2 - Plotting

#### 3.2.1 - Histograms

Creates a set of histograms by sorted data:<br>
1) Confirmed Cases:

In [ ]:
# Using the filtered plot from Stage 2 to reduce migraines (caused by the number of data points) slightly.
p2_state_hist = p2["Confirmed"].hist(bins="auto", log=True)

2) Recovered Cases:

In [ ]:
p2_chaos_hist = p2["Recovered"].hist(bins="auto", log=True)

3) Deaths:

In [ ]:
p2_balanced_hist = p2["Deaths"].hist(bins="auto", log=True)

#### 3.2.2 - Plots Within Plots

All three plots, but together now!

In [ ]:
p2_grouped_hist = p2.hist(column=["Confirmed", "Recovered", "Deaths"],
                          bins="auto", layout=(3, 1), figsize=(10, 10), log=True)

#### 3.2.3 - Single-Line Graph

This is an attempt at graphing recovered cases over time. <br>
*It turns out that 20,000 data points don't make for a good line graph.*

In [ ]:
p2["ObservationDate"] = pd.to_datetime(p2["ObservationDate"])
p2.sort_values(by="ObservationDate", ascending=True, inplace=True)
p2.plot(x="ObservationDate", y="Recovered", figsize=(10,5))

#### 3.2.4 - Multi-Line Graph

Now with less data points! <br>
Creates a function to reduce redundant code in this section somewhat.


In [ ]:
def aggravate(tmp_series, target):
    return tmp_series.aggregate({target:"last"}).groupby(["Country/Region"], as_index=False,
                                                         dropna=False).aggregate({target:"sum"})[target]

Creates a temporary *unsorted* dataframe for use with the "aggravate" function.

In [ ]:
tmp_ser = df.groupby(["Country/Region", "Province/State"], as_index=False, dropna=False)

Creates a new dataframe (lg) which is filtered to both make the data more meaningful and reduce the number of datapoints.


In [ ]:
lg = pd.DataFrame()
lg["Country"] = (tmp_ser.aggregate({"Confirmed":"last"}).groupby(["Country/Region"], as_index=False,dropna=False).
                 aggregate({"Confirmed":"sum"}))["Country/Region"]
lg["Confirmed"] = aggravate(tmp_ser, "Confirmed")
lg["Recovered"] = aggravate(tmp_ser, "Recovered")
lg["Deaths"] = aggravate(tmp_ser, "Deaths")

Creates a line graph out of the newly-filtered function using the number of confirmed cases as the x-axis.

In [ ]:
lg.sort_values(by="Confirmed", ascending=True, inplace=True)
lg.plot(
    'Confirmed', # X-Axis index
    title="COVID Case Survival Rates Over Total Cases.",
    ylabel="Cases (Log10)",
    xlabel="Confirmed Cases (Log10)",
    legend=True,
    loglog=True # Sets both axes to a logarithmic scale.
)

mp.pyplot.annotate('|---?---|', (10**7, 10**7))
mp.pyplot.annotate('|----------?----------|', (0.5, 100))

#### 3.2.5 - Bar Graph

Now with beautiful labels! <br>
Saves the artwork to a .png in the working folder.

In [ ]:
lg_bar = lg
lg_bar.set_index("Country")
lg_bar.plot.bar(
    "Country", "Confirmed", # X-Axis, Y-Axis
    title="COVID Cases by Country",
    ylabel="Confirmed Cases (Log10)",
    figsize=(50,5),
    legend=True,
    logy=True
)

mp.pyplot.savefig("not_modern_art.png")

## Stage 4 - Data Wrangling

### Part 4.1 - Hierarchical Indexing

#### Part 4.1.1 - .set_index()

- Duplicates the master dataframe into p4.
- Drops unwanted columns.
- Drops the duplicates in "Province/State", keeping the last (most recent) entry.

In [ ]:
p4 = df.copy()
p4.drop(["SNo", "ObservationDate", "Last Update"], axis=1, inplace=True) # Removes the redundant index.

p4.drop_duplicates(["Country/Region", "Province/State"], keep="last", inplace=True)

- Creates a hierarchical index from the country and state.<br>
- Sorts the index.

In [ ]:
p4_hier = p4.set_index(["Country/Region", "Province/State"])
p4_hier.sort_index(inplace=True)
p4_hier.index

#### Part 4.1.2 - .unstack()

Unstacks the dataframe at "Province/State" and makes a mess: <br>
I'm not sure why, but only 28 countries are listed. <br>
This intentionally makes no changes to the dataframe so that it can be used in the next section.

In [ ]:
p4_hier.unstack(level="Country/Region", fill_value=" ")

Earlier, .unstack() resulted in: <br>
"MemoryError: Unable to allocate 3.88 GiB for an array with shape (235243, 2214) and data type object"<br>
I assume GitHub/CoLab would not appreciate that.

#### Part 4.1.3 - Summary Statistics

Uses the hierarchical index and .sum() to find the total number of confirmed cases/ deaths/ recoveries.

In [ ]:
p4_hier.sum(level="Country/Region")

#### Part 4.1.4 - .pivot()

- Creates a pivot group by Country/Region and Province/State.
- .fillna() managed to crash pycharm then freeze my computer.

In [ ]:
p4_pivot = p4.pivot("Country/Region", "Province/State")

# p4_pivot.fillna(" ", inplace=True)
p4_pivot

This results in an identical copy of the unstacked table, as pivots are just a more concise way of using
.set_index().unstack()
The value arg of .pivot() does not need to be set, as the unwanted columns were dropped from p4.
This method is much more concise, but allows for less options than unstack.

#### Part 4.1.5 - .groupby() -> Unstack

.groupby was used several times throughout the project to make the data more useful.<br>



In [ ]:
p4_group = df.groupby(["Country/Region", "Province/State"], dropna=False).aggregate({"Confirmed":"last"})
p4_group = p4_group.unstack("Province/State", fill_value=" ")

p4_group

This condenses the data down in a similar way to the above pivot/set_index methods.


### Part 4.2 - Merge and Concat

#### Part 4.2.1a - Splitting the Dataset

Creates two dataframes and prepares them by setting a common index, then dropping various columns.<br>
Both sets drop "Last Update".

In [ ]:
p4a = df.copy()
p4b = p4.copy()

p4a.drop(["SNo", "ObservationDate", "Last Update", "Deaths", "Recovered"], axis=1, inplace=True)
p4b.drop(["Country/Region", "Province/State"], axis=1, inplace=True)
p4b

p4b is based on p4, which had all duplicate Country/Province entries dropped. P4a is a copy of the original dataframe.


#### Part 4.2.1b - pd.concat()

Combines p4a and p4b together again.

In [ ]:
p4c = pd.concat([p4a, p4b], axis=1)
p4c

The duplicate entries still carry their values for Country/Province(p4a), but show as NaN for
Confirmed/Deaths/Recovered (p4b).<br>
This is because the two tables were joined on the index automatically (with the shared column merged).

#### Part 4.2.2 - pd.merge()


##### Part 4.2.2.1 - The "On" Parameters

Merges on the shared column, "Confirmed".

In [ ]:
p4a.merge(p4b, on="Confirmed")

Somehow this ended up with more entries than it started wtih.

##### Part 4.2.2.2 - The "How" Parameter

Merges with how="left".

In [ ]:
p4a.merge(p4b, how="left", left_index=True, right_index=True)

Merges with how="right"

In [ ]:
p4a.merge(p4b, how="right", left_index=True, right_index=True)

These two tables differ in what slice was used to anchor the table- for "left" (specifying p4a), all rows are displayed,
while with "right", only the 970 rows of p4b are displayed. Both confirmed columns are displayed, as they were merged
on the index.

### Part 4.3 - Pivot Table and CrossTab

#### Part 4.3.1 - .pivot_table()

##### Part 4.3.1.1 - Replicating Hierarchical Indexing

Creates a pivot table based off of the filtered dataframe from before.<br>
Country/Region is set as the index, Province/State as the columns, and Confirmed/Deaths/Recovered as the data points.<br>
I'm not quite sure where most of the countries went, though.

In [ ]:
piv = df.copy()

piv.pivot_table(["Confirmed", "Deaths", "Recovered"], index="Country/Region", columns="Province/State", dropna=False)

###### Part 4.3.1.2 - Again, with "margins=True"

Second time's the charm.

In [ ]:
piv.pivot_table(["Confirmed", "Deaths", "Recovered"], index="Country/Region", columns="Province/State", margins=True,
                dropna=False)

Margins creates a totals row on the bottom and right side of the table.<br>
The bottom is not terribly helpful, as there is only ever one entry per column, but the right total should give the
total for the country.

###### Part 4.3.1.3 - Yet Again, but with "aggfunc="

Setting aggfunc to count counts the number of entries per city, if you can find it in the sea of NaNs.

In [ ]:
piv.pivot_table(["Confirmed", "Deaths", "Recovered"], index="Country/Region", columns="Province/State", margins=True,
                dropna=False, aggfunc="count")

#### Part 4.3.2 - pd.crosstab()

Creates a crosstab pivot table which counts the number of entries per city.

In [ ]:
# Renaming these columns makes things easier.
piv.rename(columns = {"Country/Region": "Country", "Province/State":"Province"}, inplace=True)

pd.crosstab(piv.Country, piv.Province, margins=True)

Essentially the same outcome as pivot_table with aggfunc="count".<br>
This one doesn't hurt my eyes (as much), though.

### Part 4.4 - Screencast Video

*Coming Soon...*
